In [2]:
import json
import os
import re

# Function to extract mapping entries from a JSON file
def extract_mapping_entries(filename):
    match = re.search(r'JSON_([a-zA-Z]+)_(\d{4})_AC_(\d{2}).json', filename)
    # match = re.search(r'JSON_([a-zA-Z]+)_(\d{4})_AC_(AC\d{2})Form\d{2}\.json',filename)
    # match = re.search(r'JSON_([a-zA-Z]+)_LokSabha_Election_(\d{4})_AC_(\d{3}).json', filename)

    # match = re.search(r"Form20-(\d+)", filename)
    # match = re.search(r'JSON_([a-zA-Z]+)_(\d{4})_AC_(AC\d{2})Form\d{2}\.json',filename)
    
    if match:
        # state = match.group(1).lower()  # Convert state name to lowercase
        # state = match.group(1).lower()
        state = 'chhattisgarh'
        year = int(match.group(2))
        AC = int(match.group(3))
    else:
        print(f"Filename {filename} does not match expected format.")
        return []

    with open(filename) as f:
        data = json.load(f)

    table = data['tables'][0]
    col_headers = []
    cells = table.get('cells')

    for i in range(1,table.get('column_count')):
        col_cells = [cell for cell in cells if cell.get('column_index') == i and cell.get('kind') == 'columnHeader' and cell.get('column_span') == 1]
                
        # contents = [cell.get('content') for cell in col_cells]
        # unwanted_terms = ["NOTA", "None", "station", "polling", "total", "vote", "rendered", "rejected", "valid", "serial"]
        unwanted_terms = ["मतदान","केन्द्र","क्रम","संख्या","विधिमान्य","मतों","नोटा","एनओटीए","निविदत्त","कुल","NOTA","अस्वीकृत","स.क.","इनमें" ,"कोई","नही","पार्टी","बहुजन","मुक्ति","समाज","इंडियन","नेशनल","कौंग्रेस","छत्तीसगढ़","स्वाभिमान","मंच","आबडकराइट","ऑफ","आम","आदमी","निर्दलीय","बहुजन","(भा.ज.पा.)","(ब.स.पा.)","(इं.ने.कां.)","(आ.आ.पा.)","(जद यू.)","(निर्दलीय)","विकल्प"]

        # Filtering out the cells whose content contains any of the unwanted terms
        contents = [cell.get('content') for cell in col_cells if not any(term.lower() in (cell.get('content') or "").replace(" ", "").replace("\n","").lower() for term in unwanted_terms)]
  
        # contents = [content for content in contents if re.search(r'[a-zA-Z0-9]', content) and len(content) > 4]
        contents = [content for content in contents if re.search(r'[\u0900-\u097F]', content) and len(content) > 2]

        content = contents[0] if len(contents) else None
        if content: 
            col_headers.append({
                "name": content.replace('\n', ' ').strip(),
                "index" : i + 1 
            })

    mapping_entries = []
    for column in col_headers:
        name = column.get('name')
        # name = re.sub(r'[^\x00-\x7F]+', '', name)
        name = re.sub(r'[^ऀ-ॿ\s]+', '', name)
        mapping_entry = {
            "state": state,
            "year": year,
            "AC": AC,
            "name": name,
            "column": column.get('index'),
        }
        mapping_entries.append(mapping_entry)

    return mapping_entries


In [3]:
# folder_path = 'results/Parsed_Pdfs/RA_filtered/2023'

folder_path = 'results/Parsed_Pdfs/Chhattisgarh/AE_2018'
# folder_path = 'results/Parsed_Pdfs/Maharastra/Lok Sabha Election 2019'
final_mapping_entries = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)

        mapping_entries = extract_mapping_entries(file_path)
        final_mapping_entries.extend(mapping_entries)

state = 'chhattisgarh'
year = 2018
# type = 'GA'
type = 'AE'
# Write final mapping entries to a JSON file
final_mapping_filename = f'results/name_mappings/{state}_{year}_{type}.json'

with open(final_mapping_filename, 'w') as f:
    json.dump(final_mapping_entries, f, indent=4)

print(f"Final mapping file created: {final_mapping_filename}")

Final mapping file created: results/name_mappings/chhattisgarh_2018_AE.json


In [4]:
import json

# Read the existing JSON file
with open('results/name_mappings/chhattisgarh_2018_AE.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Write the data back to a new JSON file with literal Devanagari characters
with open('results/name_mappings/chhattisgarh_2018_AE_hindi_casted.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


In [ ]:
import openai

openai.api_key="sk-proj-QWTGhoeV4F4nG3y5v8WhT3BlbkFJCwQRD596VdofAMyqkm4q"

name_translations = {
    "John": "जॉन",
    "Mary": "मेरी",
    "David": "डेविड",
    "Sarah": "सारा"
}

def get_model_response(model, user_prompt, prompt, temperature=0, logprobs=False, top_logprobs=None, response_format=None):
    '''
    Function to get model response
    '''
    params = {
        "model": model,
        "temperature": temperature,
        "logprobs": logprobs,
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_prompt}
        ]
    }
    if response_format:
        params["response_format"] = response_format

    if logprobs:
        params["top_logprobs"] = top_logprobs if top_logprobs else 1

    response = openai.chat.completions.create(
        **params
    )
    return response